In [ ]:
# for google colaboratory
from os import path
if 'google.colab' in str(get_ipython()):
    files = ['LLD-icon-sharp.hdf5']
    !pip install wget
    !pip install transformers
    import wget
    import shutil
    from google.colab import drive
    drive.mount('/content/drive')
    !mkdir Data
    for f in files:
        if not path.isfile('Data/' + f):
            if path.isfile('/content/drive/My Drive/Colab/AFRO/' + f):
                shutil.copy('/content/drive/My Drive/Colab/AFRO/' + f, 'Data')
            else:
                wget.download('https://data.vision.ee.ethz.ch/sagea/lld/data/' + f, 'Data')

In [ ]:
# for debugging ------
from importlib import reload
import utils
utils = reload(utils)
from utils import lemmatize_and_clearing, embed_and_write_file
# -------------------
import pickle

import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from transformers import BertTokenizer, BertModel

import nltk

import torch
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Open data

## icon-sharp

In [ ]:
files = h5py.File('Data/LLD-icon-sharp.hdf5', 'r')

In [ ]:
files.keys()

In [ ]:
clusters = files['labels/resnet/rc_128']
names = files['meta_data/names'][()].astype(str)
images = files['data']

# Work with labels

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
lem_names =  map(lemmatize_and_clearing, names)

## Download bert files and berting all that is possible

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
tok_names = tokenizer(list(lem_names), padding=True, return_tensors="pt")

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
model = model.to(device)

In [ ]:
batch_size = 10
dataloader = DataLoader(tok_names['input_ids'], batch_size=batch_size)

In [ ]:
embed_and_write_file(dataloader, model, device, 'embed_train.csv')

# Create text and add to icon

In [ ]:
import albumentations as A
import cv2 as cv
# custom module --- 
import utils
utils = reload(utils)
from utils import add_text_to_img
# ------------------

In [ ]:
from skimage.transform import resize
size = (128, 128)
img = resize(np.moveaxis(images[2], 0, -1), size, mode='constant', anti_aliasing=True,)


In [ ]:
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
plt.imshow(add_text_to_img('Hello', img))


In [ ]:
# Подготовка данных

# Создание и тренировка модели

In [ ]:
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)


In [ ]:
gen = Generator(100, 3)
disc = Discriminator((32, 32, 3))

In [ ]:
gan = WGAN(discriminator=disc, generator=gen, latent_dim=100)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

In [ ]:
gan.fit(dataset.take(100), epochs=1)